In [13]:
import boa
from dataclasses import dataclass
from rich.console import Console as RichConsole

from boa.integrations.jupyter import BrowserSigner
from boa.network import NetworkEnv

In [15]:
factories = {
    # Ethereum
    "ethereum:sepolia": {
        "factory": "0x98EE851a00abeE0d95D08cF4CA2BdCE32aeaAF7F",
    },
    "ethereum:mainnet": {
        "factory": "0x98EE851a00abeE0d95D08cF4CA2BdCE32aeaAF7F",
    },
    # Layer 2
    "arbitrum:mainnet": {
        "factory": "0x98EE851a00abeE0d95D08cF4CA2BdCE32aeaAF7F",
    },
    "optimism:mainnet": {
        "factory": "0x98EE851a00abeE0d95D08cF4CA2BdCE32aeaAF7F",
    },
    "base:mainnet": {
        "factory": "0xc9Fe0C63Af9A39402e8a5514f9c43Af0322b665F",
    },
    "linea:mainnet": {
        "factory": "0x98EE851a00abeE0d95D08cF4CA2BdCE32aeaAF7F",
    },
    "scroll:mainnet": {
        "factory": "0x98EE851a00abeE0d95D08cF4CA2BdCE32aeaAF7F",
    },
    "zksync:mainnet": {
        "factory": "",
    },
    "pzkevm:mainnet": {
        "factory": "0x98EE851a00abeE0d95D08cF4CA2BdCE32aeaAF7F",
    },
    "mantle:mainnet": {"factory": "0x98EE851a00abeE0d95D08cF4CA2BdCE32aeaAF7F"},
    # Layer 1
    "gnosis:mainnet": {
        "factory": "0x98EE851a00abeE0d95D08cF4CA2BdCE32aeaAF7F",
    },
    "polygon:mainnet": {
        "factory": "0x98EE851a00abeE0d95D08cF4CA2BdCE32aeaAF7F",
    },
    "avax:mainnet": {
        "factory": "0x98EE851a00abeE0d95D08cF4CA2BdCE32aeaAF7F",
    },
    "ftm:mainnet": {
        "factory": "0x98EE851a00abeE0d95D08cF4CA2BdCE32aeaAF7F",
    },
    "bsc:mainnet": {
        "factory": "0x98EE851a00abeE0d95D08cF4CA2BdCE32aeaAF7F",
    },
    "celo:mainnet": {
        "factory": "0x98EE851a00abeE0d95D08cF4CA2BdCE32aeaAF7F",
    },
    "kava:mainnet": {
        "factory": "0xd3B17f862956464ae4403cCF829CE69199856e1e",
    },
    "aurora:mainnet": {
        "factory": "0x98EE851a00abeE0d95D08cF4CA2BdCE32aeaAF7F",
    },
    "tron:mainnet": {
        "factory": "",
    },
}

In [16]:
pool_settings = [
    'CRV/ETH',
    'crv-eth',
    ['0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', '0xD533a949740bb3306d119CC777fa900bA034cd52'],
    # ['0x498Bf2B1e120FeD3ad3D42EA2165E9b73f99C1e5', '0x11cDb42B0EB46D95f990BeDD4695A6e3fA034978'],
    0,
    2700000,
    1300000000000,
    2999999,
    80000000,
    350000000000000,
    100000000000,
    100000000000,
    866,
    10**18
]

In [17]:
def deploy_pool(network, url, fork):

    if fork:
        boa.env.fork(url)
        print("Deploying a pool in forkmode...")
        boa.env.eoa = "0x3ee18B2214AFF97000D974cf647E7C347E8fa585"
    
    else:
        print("Deploying a pool in production...")
        boa.set_env(NetworkEnv(url))
        boa.env.add_account(BrowserSigner())

    factory_address = factories[network]["factory"]
    if not factory_address:
        print(f"No factory address found for {network}.")
        return


    factory = boa.load_partial("../contracts/main/CurveTwocryptoFactory.vy")
    factory = factory.at(factory_address)

    amm_address = factory.deploy_pool(*pool_settings)

    # import pool implementation to check parameters
    pool_impl = boa.load_partial("../contracts/main/CurveTwocryptoOptimized.vy")
    pool = pool_impl.at(amm_address)

    print("Pool deployed at:", amm_address)
    print("-------------------")
    print("Name:", pool.name())
    print("Symbol:", pool.symbol())
    print("Coin 0:", pool.coins(0))
    print("Coin 1:", pool.coins(1))
    print("A:", pool.A())
    print("Gamma:", pool.gamma())
    print("Mid Fee:", pool.mid_fee())
    print("Out Fee:", pool.out_fee())
    print("Fee Gamma:", pool.fee_gamma())
    print("Allowed Extra Profit:", pool.allowed_extra_profit())
    print("Adjustment Step:", pool.adjustment_step())
    print("MA Exp Time:", pool.ma_time())

    return amm_address

In [18]:
chain = "ethereum:mainnet"
url = "https://eth-mainnet.g.alchemy.com/v2/{API_KEY}"
fork = True

deploy_pool(chain, url, fork)

Deploying a pool in forkmode...


HTTPError: 401 Client Error: Unauthorized for url: https://eth-mainnet.g.alchemy.com/v2/API_KEY